In [1]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50

In [3]:
# =============== 1. Preparation of Device, Model, and Transform ===============
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume the number of output classes during training was 9 (including background)
num_classes = 9

# 1.1 Define the network and load local weights
model = deeplabv3_resnet50(num_classes=num_classes)
# If you saved only the state_dict, use load_state_dict
# If you encounter an error, it may mean the entire model was saved, and you need to use torch.load() instead
model.load_state_dict(
    torch.load("/content/deeplabv3_finetuned_RS_openearthmap_v2.pth", map_location=device),
    strict=False
)

model.to(device)
model.eval()  # Switch to evaluation mode

# 1.2 Define the image transformation for inference (consistent with training)
transform = transforms.Compose([
    # This is just an example: only converting the image to a Tensor.
    # If you performed Resize/Crop/Normalize during training, ensure consistency here
    transforms.ToTensor()
])

<ipython-input-3-2ae5645cc312>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load("/content/deeplabv3_finetuned_RS_openearthmap_v2.pth", map_location=device),


In [25]:
# =============== 2. Specify Input and Output Directories ===============
input_dir = "./House_price/satellite_images/SanJose"  # Folder containing images for inference
#output_dir = "./out"
output_dir = "./SanJose"
os.makedirs(output_dir, exist_ok=True)

# =============== 3. Define the Color Palette idx2color Corresponding to Your Training ===============
# Example: 0=background, 1=Bareland, 2=Rangeland, 3=Developed space, 4=Road,
# 5=Tree, 6=Water, 7=Agriculture land, 8=Building
idx2color = np.array([
    [0,   0,   0  ],  # 0: background
    [128, 0,   0  ],  # 1=Bareland
    [0,   255, 36 ],  # 2=Rangeland
    [148, 148, 148],  # 3=Developed space
    [255, 255, 255],  # 4=Road
    [34,  97,  38 ],  # 5=Tree
    [0,   69,  255],  # 6=Water
    [75,  181, 73 ],  # 7=Agriculture land
    [222, 31,  7  ],  # 8=Building
], dtype=np.uint8)

# (Optional) If you need to visualize the legend, you can keep this dictionary:
color_mapping = {
    "Bareland":        [128, 0,   0],
    "Rangeland":       [0,   255, 36],
    "Developed space": [148, 148, 148],
    "Road":            [255, 255, 255],
    "Tree":            [34,  97,  38],
    "Water":           [0,   69,  255],
    "Agriculture":     [75,  181, 73],
    "Building":        [222, 31,  7],
}

In [26]:
from tqdm import tqdm
# =============== 4. Batch Process All Images in the Specified Folder ===============
valid_exts = {".jpg", ".png", ".jpeg"}

# Get a list of valid image files in the input directory
image_files = [file_name for file_name in os.listdir(input_dir) if os.path.splitext(file_name)[1].lower() in valid_exts]

# Create a progress bar for the list of images
for file_name in tqdm(image_files, desc="Processing Images", unit="image"):
    img_path = os.path.join(input_dir, file_name)
    print(f"Inferencing on: {img_path}")

    # ============== 4.1. Read and Transform ==============
    input_image = Image.open(img_path).convert("RGB")
    input_tensor = transform(input_image).unsqueeze(0).to(device)  # shape: [1, 3, H, W]

    # ============== 4.2. Forward Inference ==============
    with torch.no_grad():
        outputs = model(input_tensor)          # dict: {"out": logits, ...}
        logits = outputs["out"]               # shape: [1, num_classes, H, W]
        pred = logits.argmax(dim=1)           # [1, H, W]
        pred = pred.squeeze(0).cpu().numpy()  # (H, W)

    # ============== 4.3. Map Prediction to a Colored Image ==============
    # Classes outside the defined range are treated as background
    pred[pred >= len(idx2color)] = 0
    pred_color = idx2color[pred]  # (H, W, 3)
    pred_image = Image.fromarray(pred_color)

    # ============== 4.4. Save Inference Results to output_dir ==============
    save_path = os.path.join(output_dir, file_name)
    pred_image.save(save_path)
    print(f"Saved predicted segmentation to: {save_path}")

Processing Images:   0%|          | 4/2837 [00:00<01:24, 33.34image/s]

Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.225352_-121.874203.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.225352_-121.874203.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.351505_-121.834935.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.351505_-121.834935.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.284913_-121.8174.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.284913_-121.8174.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.302256_-121.952355.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.302256_-121.952355.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.322176_-121.904552.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.322176_-121.904552.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.235347_-1

Processing Images:   0%|          | 12/2837 [00:00<01:24, 33.62image/s]

Saved predicted segmentation to: /content/SanJose/satellite_37.388802_-121.822809.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.409723_-121.862903.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.409723_-121.862903.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.264275_-121.810606.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.264275_-121.810606.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.231185_-121.882442.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.231185_-121.882442.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.303498_-121.936117.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.303498_-121.936117.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.287952_-121.759085.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.287952_-121.75908

Processing Images:   1%|          | 20/2837 [00:00<01:22, 33.96image/s]

...

Saved predicted segmentation to: /content/SanJose/satellite_37.314503_-121.902522.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.342457_-121.83875.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.342457_-121.83875.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.271414_-121.847668.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.271414_-121.847668.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.262844_-121.828906.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.262844_-121.828906.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.314677_-121.794686.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.314677_-121.794686.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.31632_-121.891636.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.31632_-121.891636.jp

Processing Images: 100%|█████████▉| 2828/2837 [01:23<00:00, 33.39image/s]

Saved predicted segmentation to: /content/SanJose/satellite_37.316975_-121.983489.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.259998_-121.851277.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.259998_-121.851277.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.194261_-121.829916.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.194261_-121.829916.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.293567_-121.883408.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.293567_-121.883408.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.38761_-121.878024.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.38761_-121.878024.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.255536_-121.841821.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.255536_-121.841821.

Processing Images: 100%|██████████| 2837/2837 [01:23<00:00, 34.05image/s]

Saved predicted segmentation to: /content/SanJose/satellite_37.264321_-121.796397.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.350554_-121.80087.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.350554_-121.80087.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.276373_-121.808562.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.276373_-121.808562.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.322058_-121.814459.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.322058_-121.814459.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.415863_-121.863665.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.415863_-121.863665.jpg
Inferencing on: /content/House_price/satellite_images/SanJose/satellite_37.258794_-121.806536.jpg
Saved predicted segmentation to: /content/SanJose/satellite_37.258794_-121.806536.

In [9]:
!git clone https://huggingface.co/datasets/Zongrong/House_price

Cloning into 'House_price'...
remote: Enumerating objects: 29510, done.
remote: Counting objects: 100% (29507/29507), done.
remote: Compressing objects: 100% (29500/29500), done.
remote: Total 29510 (delta 20), reused 29482 (delta 5), pack-reused 3 (from 1)
Receiving objects: 100% (29510/29510), 4.77 MiB | 7.12 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (29468/29468), done.
Filtering content: 100% (29460/29460), 2.67 GiB | 2.61 MiB/s, done.


In [27]:
import os
import zipfile

def zip_folder(folder_path, output_zip_path):
    """
    Compress a folder into a .zip file.

    :param folder_path: Path to the folder to be compressed.
    :param output_zip_path: Path where the .zip file will be saved.
    """
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Get the full path of the file
                file_path = os.path.join(root, file)
                # Get the relative path to avoid storing absolute paths in the zip
                arcname = os.path.relpath(file_path, folder_path)
                # Add the file to the zip
                zipf.write(file_path, arcname)

# Example usage
folder_to_zip = "./SanJose"  # Replace with the folder you want to zip
output_zip_file = "SanJose.zip"  # Replace with the desired output .zip file name

zip_folder(folder_to_zip, output_zip_file)

print(f"Folder '{folder_to_zip}' has been compressed into '{output_zip_file}'.")


Folder '/content/SanJose' has been compressed into 'SanJose.zip'.
